## Simulação de uma LLM com memória da conversa (versão mais moderna do framework e com sumário de conversa)

ref.: 
* https://python.langchain.com/docs/how_to/message_history/
* https://www.langchain.com/langgraph
* https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/

**todo**:
- Tentar gerenciar a memória da conversa de forma automatizada

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import Literal

## Utilização do llama3.1 com interface via ollama

In [2]:
# Inicialize o modelo ChatOllama
llm = ChatOllama(model="llama3.1:latest")

# Adicione um `summary` ao estado além das mensagens
class State(MessagesState):
    summary: str

# Defina o gráfico de estados
workflow = StateGraph(state_schema=State)

# Função que chama o modelo
def call_model(state: State):
    # Adiciona um resumo se ele existir
    summary = state.get("summary", "")
    if summary:
        system_message = f"Resumo da conversa anterior: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    
    response = llm.invoke(messages)
    return {"messages": [response]}

# Função que decide quando resumir a conversa
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    if len(state["messages"]) > 6:  # Se mais de 6 mensagens, sumarizar
        return "summarize_conversation"
    return END

# Função que realiza o resumo da conversa
def summarize_conversation(state: State):
    summary = state.get("summary", "")
    if summary:
        summary_message = (
            f"Este é o resumo da conversa até o momento: {summary}\n\n"
            "Estenda o resumo considerando as novas mensagens acima:"
        )
    else:
        summary_message = "Crie um resumo da conversa acima:"
    
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    
    # Exclui mensagens antigas, mantendo apenas as últimas duas
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Configuração do gráfico de estados
workflow.add_node("conversation", call_model)
workflow.add_node("summarize_conversation", summarize_conversation)

workflow.add_edge(START, "conversation")

workflow.add_conditional_edges(
    "conversation",
    should_continue,
)

workflow.add_edge("summarize_conversation", END)

# Inicialize a memória
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Função para exibir as atualizações
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

# Configuração de thread
config = {"configurable": {"thread_id": "abc123"}}

## Pergunta para iniciar a conversa

In [3]:
config = {"configurable": {"thread_id": "abc123"}}

query = "Quem foi Albert Einstein?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Quem foi Albert Einstein?
================================== Ai Message ==================================

O físico alemão, mais conhecido por seu contributo para o desenvolvimento da teoria da relatividade.


In [4]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='2a77269d-9586-4fba-b3c8-5f325fef0166'),
  AIMessage(content='O físico alemão, mais conhecido por seu contributo para o desenvolvimento da teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:03.052475871Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1993552782, 'load_duration': 11239315, 'prompt_eval_count': 16, 'prompt_eval_duration': 405975000, 'eval_count': 26, 'eval_duration': 1535052000}, id='run-a57a6bd9-39b8-47af-8d0b-ec1715579375-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42})]}

## O modelo consegue se lembrar do que acabou de responder

In [5]:
query = "O que você disse?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

O que você disse?
================================== Ai Message ==================================

Peço desculpas pelo acréscimo! A resposta simples é: Albert Einstein foi um dos cientistas mais importantes do século XX, reconhecido como o pai da física moderna. Ele desenvolveu a Teoria da Relatividade e recebeu o Prêmio Nobel de Física em 1921 por sua contribuição para a mecânica quântica.


In [6]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='2a77269d-9586-4fba-b3c8-5f325fef0166'),
  AIMessage(content='O físico alemão, mais conhecido por seu contributo para o desenvolvimento da teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:03.052475871Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1993552782, 'load_duration': 11239315, 'prompt_eval_count': 16, 'prompt_eval_duration': 405975000, 'eval_count': 26, 'eval_duration': 1535052000}, id='run-a57a6bd9-39b8-47af-8d0b-ec1715579375-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42}),
  HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='47204337-5db4-42b5-b53e-8a482d21295b'),
  AIMessage(content='Peço desculpas pelo acréscimo! A resposta simples é: Albert Einstein foi um

In [7]:
query = "O que você disse? Ainda não consegui entender"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

O que você disse? Ainda não consegui entender
================================== Ai Message ==================================

Não se preocupe! Vou explicar melhor.

Albert Einstein foi um físico (cientista que estuda a matéria e a energia) muito importante. Ele viveu no século XX e fez descobertas incríveis sobre o universo.

Algumas das coisas mais importantes que ele descobriu incluem:

* A Teoria da Relatividade: uma ideia que mostra como o tempo e o espaço são relacionados.
* A famosa equação E=mc²: que mostra que a energia (E) é igual à massa (m) multiplicada pela velocidade da luz ao quadrado (c).

Ele também recebeu um prêmio muito importante, chamado Prêmio Nobel, por suas descobertas.

Einstein é considerado um dos cientistas mais importantes de todos os tempos!


In [8]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Quem foi Albert Einstein?', additional_kwargs={}, response_metadata={}, id='2a77269d-9586-4fba-b3c8-5f325fef0166'),
  AIMessage(content='O físico alemão, mais conhecido por seu contributo para o desenvolvimento da teoria da relatividade.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:03.052475871Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1993552782, 'load_duration': 11239315, 'prompt_eval_count': 16, 'prompt_eval_duration': 405975000, 'eval_count': 26, 'eval_duration': 1535052000}, id='run-a57a6bd9-39b8-47af-8d0b-ec1715579375-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42}),
  HumanMessage(content='O que você disse?', additional_kwargs={}, response_metadata={}, id='47204337-5db4-42b5-b53e-8a482d21295b'),
  AIMessage(content='Peço desculpas pelo acréscimo! A resposta simples é: Albert Einstein foi um

In [9]:
query = "Nasceu onde?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Nasceu onde?
================================== Ai Message ==================================

Albert Einstein nasceu em 14 de março de 1879 em Ulma, no Reino da Alemanha (atualmente na Alemanha).
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


Aqui está o resumo:

**Conversa sobre Albert Einstein**

* O que foi dito:
 + Albert Einstein foi um dos cientistas mais importantes do século XX.
 + Ele desenvolveu a Teoria da Relatividade e recebeu o Prêmio Nobel de Físi

In [10]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Nasceu onde?', additional_kwargs={}, response_metadata={}, id='a32e5485-2693-4531-9dd9-f685bf491f25'),
  AIMessage(content='Albert Einstein nasceu em 14 de março de 1879 em Ulma, no Reino da Alemanha (atualmente na Alemanha).', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:25.637938011Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5701073488, 'load_duration': 11612920, 'prompt_eval_count': 349, 'prompt_eval_duration': 483096000, 'eval_count': 35, 'eval_duration': 2121982000}, id='run-71972bea-28a3-4c59-bfd3-c9a14ac95ac1-0', usage_metadata={'input_tokens': 349, 'output_tokens': 35, 'total_tokens': 384})],
 'summary': 'Aqui está o resumo:\n\n**Conversa sobre Albert Einstein**\n\n* O que foi dito:\n + Albert Einstein foi um dos cientistas mais importantes do século XX.\n + Ele desenvolveu a Teoria da Relatividade e recebeu o Prêmio Nobel d

In [11]:
query = "Quando é o aniversário dele?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Quando é o aniversário dele?
================================== Ai Message ==================================

O aniversário do Albert Einstein é em 14 de março.


In [12]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Nasceu onde?', additional_kwargs={}, response_metadata={}, id='a32e5485-2693-4531-9dd9-f685bf491f25'),
  AIMessage(content='Albert Einstein nasceu em 14 de março de 1879 em Ulma, no Reino da Alemanha (atualmente na Alemanha).', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:25.637938011Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5701073488, 'load_duration': 11612920, 'prompt_eval_count': 349, 'prompt_eval_duration': 483096000, 'eval_count': 35, 'eval_duration': 2121982000}, id='run-71972bea-28a3-4c59-bfd3-c9a14ac95ac1-0', usage_metadata={'input_tokens': 349, 'output_tokens': 35, 'total_tokens': 384}),
  HumanMessage(content='Quando é o aniversário dele?', additional_kwargs={}, response_metadata={}, id='e2e7b5d2-77fa-410b-8f23-8aa59382e8bd'),
  AIMessage(content='O aniversário do Albert Einstein é em 14 de março.', additional_kwargs={}

In [13]:
query = "O que você disse por último?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

O que você disse por último?
================================== Ai Message ==================================

Disse que o aniversário do Albert Einstein é em 14 de março, ou seja, seu nascimento ocorreu no dia 14 de março.


In [14]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Nasceu onde?', additional_kwargs={}, response_metadata={}, id='a32e5485-2693-4531-9dd9-f685bf491f25'),
  AIMessage(content='Albert Einstein nasceu em 14 de março de 1879 em Ulma, no Reino da Alemanha (atualmente na Alemanha).', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:25.637938011Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5701073488, 'load_duration': 11612920, 'prompt_eval_count': 349, 'prompt_eval_duration': 483096000, 'eval_count': 35, 'eval_duration': 2121982000}, id='run-71972bea-28a3-4c59-bfd3-c9a14ac95ac1-0', usage_metadata={'input_tokens': 349, 'output_tokens': 35, 'total_tokens': 384}),
  HumanMessage(content='Quando é o aniversário dele?', additional_kwargs={}, response_metadata={}, id='e2e7b5d2-77fa-410b-8f23-8aa59382e8bd'),
  AIMessage(content='O aniversário do Albert Einstein é em 14 de março.', additional_kwargs={}

In [15]:
query = "Qual foi a maior contribuição dele?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Qual foi a maior contribuição dele?
================================== Ai Message ==================================

A maior contribuição de Albert Einstein é ter desenvolvido a Teoria da Relatividade, que revolucionou o entendimento da física e do espaço-tempo. Uma das equações mais famosas dela é E=mc².
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


Aqui está o resumo atualizado:

**Conversa sobre Albert Einstein**

* O que foi dito:
 + Albert Einstein nasceu 

In [16]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Qual foi a maior contribuição dele?', additional_kwargs={}, response_metadata={}, id='4158d5f0-6fd7-421e-91d0-90a0cee3aa51'),
  AIMessage(content='A maior contribuição de Albert Einstein é ter desenvolvido a Teoria da Relatividade, que revolucionou o entendimento da física e do espaço-tempo. Uma das equações mais famosas dela é E=mc².', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:42.629319115Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3820325466, 'load_duration': 61772585, 'prompt_eval_count': 281, 'prompt_eval_duration': 190885000, 'eval_count': 51, 'eval_duration': 3096398000}, id='run-909222f9-ef36-4b43-9a6f-bc61843285d5-0', usage_metadata={'input_tokens': 281, 'output_tokens': 51, 'total_tokens': 332})],
 'summary': 'Aqui está o resumo atualizado:\n\n**Conversa sobre Albert Einstein**\n\n* O que foi dito:\n + Albert Einstein na

In [17]:
query = "O que você disse por último?"
input_message = HumanMessage(content=query)
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

O que você disse por último?
================================== Ai Message ==================================

Eu disse que a maior contribuição de Albert Einstein foi ter desenvolvido a Teoria da Relatividade, e uma das equações mais famosas dela é E=mc².


In [18]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Qual foi a maior contribuição dele?', additional_kwargs={}, response_metadata={}, id='4158d5f0-6fd7-421e-91d0-90a0cee3aa51'),
  AIMessage(content='A maior contribuição de Albert Einstein é ter desenvolvido a Teoria da Relatividade, que revolucionou o entendimento da física e do espaço-tempo. Uma das equações mais famosas dela é E=mc².', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-10-05T12:15:42.629319115Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3820325466, 'load_duration': 61772585, 'prompt_eval_count': 281, 'prompt_eval_duration': 190885000, 'eval_count': 51, 'eval_duration': 3096398000}, id='run-909222f9-ef36-4b43-9a6f-bc61843285d5-0', usage_metadata={'input_tokens': 281, 'output_tokens': 51, 'total_tokens': 332}),
  HumanMessage(content='O que você disse por último?', additional_kwargs={}, response_metadata={}, id='88e8a026-8f1c-40f4-a2d7